## This code extract the urls for some 10-k and 10-q of interest harnessing the advantages of SEC API

In [1]:
# Libraries
import pandas as pd
import requests
import time
import json
from tqdm import tqdm

In [2]:
# Set the headers to use with the API
# You must put your email or you can't retrieve anything
headers = {'User-Agent': 'salazarc2@duq.edu'}

In [3]:
# This dictionary contain a sample of companies whose 10 form will be retrieved
"""
    metadata:
    form_type = type of document, 10K, 10Q, 8K, etc.
    filling_date = date when the company send the filing to the SEC
    reporting_date = accounting period to report, end of quarter for 10Q, end of year for 10K
    ticker = unique identifier of the company in the stock market
"""


data = {
    
    "form_type": [
        "10-Q", "10-Q", "10-K", "10-Q", "10-K", 
        "10-Q", "10-Q", "10-Q", "10-k", "10-K", 
        "10-Q", "10-Q", "10-Q", "10-Q", "10-Q"
    ],
    "filing_date": [
        "4/26/2022", "8/4/2017", "3/10/2017", "5/3/2017", "2/22/2017", 
        "2/8/2017", "5/9/2016", "11/4/2016", "2/16/2017", "7/30/2018", 
        "8/5/2016", "11/9/2016", "11/9/2016", "11/2/2017", "8/4/2016"
    ],
    "reporting_date": [
        "3/31/2022", "6/30/2017", "12/31/2016", "3/31/2017", "12/31/2016", 
        "12/31/2016", "3/31/2016", "10/1/2016", "12/31/2016", "12/31/2016", 
        "6/30/2016", "9/30/2016", "9/30/2016", "9/30/2017", "6/25/2016"
    ],
    "ticker": [
        "MSFT", "CNSL", "AMC", "ABT", "ARCC", 
        "MCHP", "OCFC", "TMO", "ONB", "HBAN", 
        "PNFP", "WLK", "D", "ABT", "FORM"
    ],
}

In [4]:
# Convert the dictionary to a data frame and completing the cik, ciks have 10 digits
# So we need to fill the leading zeroes to complete the ciks
companies_df = pd.DataFrame(data)
# Convert 'reporting_date' to datetime format according to SEC API json files
companies_df['filing_date'] = pd.to_datetime(companies_df['filing_date']).dt.strftime('%Y-%m-%d')
companies_df['reporting_date'] = pd.to_datetime(companies_df['reporting_date']).dt.strftime('%Y-%m-%d')
companies_df.head(15)

,form_type,filing_date,reporting_date,ticker
0,10-Q,2022-04-26,2022-03-31,MSFT
1,10-Q,2017-08-04,2017-06-30,CNSL
2,10-K,2017-03-10,2016-12-31,AMC
3,10-Q,2017-05-03,2017-03-31,ABT
4,10-K,2017-02-22,2016-12-31,ARCC
5,10-Q,2017-02-08,2016-12-31,MCHP
6,10-Q,2016-05-09,2016-03-31,OCFC
7,10-Q,2016-11-04,2016-10-01,TMO
8,10-k,2017-02-16,2016-12-31,ONB
9,10-K,2018-07-30,2016-12-31,HBAN


## It is better to have the cik number of each company. The ticker is available, let's find the cik

In [5]:
# Filter the DataFrame to get a list of tickers
ticker_list = companies_df['ticker'].tolist()

In [6]:
# Get with request the json file with all the public companies ciks and tickers
# Remember to include the headers
try:
    response = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
    response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
    tickers_json = response.json()
    print("JSON was successfully retrieved.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

JSON was successfully retrieved.


In [7]:
def fetch_cik(ticker, tickers_json):
    """Arguments: Ticker of the company, dictionary with tickers
       and corresponding ciks
       Returns: The cik of the required company.       
    """
    ticker = ticker.upper() # Convert ticker to uppercase (just in case)
    for company in tickers_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10) # Turn the integer into a string and complete leading zeros
            return cik
    # Return None if ticker is not found
    return None

In [8]:
# Let's loop the list of tickers to retrieve the cik
# It is better to use a dictionary because the ticker and cik are unique
dict_cik = {}
for ticker in ticker_list:
    cik = fetch_cik(ticker, tickers_json)
    dict_cik[ticker] = cik
print(dict_cik)

{'MSFT': '0000789019', 'CNSL': '0001304421', 'AMC': '0001411579', 'ABT': '0000001800', 'ARCC': '0001287750', 'MCHP': '0000827054', 'OCFC': '0001004702', 'TMO': '0000097745', 'ONB': '0000707179', 'HBAN': '0000049196', 'PNFP': '0001115055', 'WLK': '0001262823', 'D': '0000715957', 'FORM': '0001039399'}


In [9]:
# Map the 'ticker' column to the new 'cik' column using the provided dictionary
companies_df['cik'] = companies_df['ticker'].map(dict_cik)
companies_df.head(15)

,form_type,filing_date,reporting_date,ticker,cik
0,10-Q,2022-04-26,2022-03-31,MSFT,0000789019
1,10-Q,2017-08-04,2017-06-30,CNSL,0001304421
2,10-K,2017-03-10,2016-12-31,AMC,0001411579
3,10-Q,2017-05-03,2017-03-31,ABT,0000001800
4,10-K,2017-02-22,2016-12-31,ARCC,0001287750
5,10-Q,2017-02-08,2016-12-31,MCHP,0000827054
6,10-Q,2016-05-09,2016-03-31,OCFC,0001004702
7,10-Q,2016-11-04,2016-10-01,TMO,0000097745
8,10-k,2017-02-16,2016-12-31,ONB,0000707179
9,10-K,2018-07-30,2016-12-31,HBAN,0000049196


## Let's define the functions to retrieve que 10K and 10Q documents, some documents are in 'recent' filings, other are in 'files', you need to learn about the structure of the json file

In [10]:
# Function to obtain recent submissions
def get_document_url(cik, reporting_date):
    """Arguments: Cik and effective date for each transaction
       Returns: The url to access the document directly.
    """
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    
    # Request to the API
    response = requests.get(url, headers=headers)
    
    # Check successful request
    if response.status_code == 200:
        company_json = response.json()
        
        # Create a df that countains the recent files
        df_recent = pd.DataFrame(company_json['filings']['recent'])
        df_files = pd.DataFrame(company_json['filings']['files'])
        
        # Check that the reporting date is in recent
        if 'reportDate' in df_recent.columns:
            
            # Select only the 10-K and the 10-Q case insensitive
            df_r = df_recent[df_recent['form'].str.lower().isin(['10-k', '10-q'])]
            
            if not df_r.empty and reporting_date in df_r['reportDate'].values:
                
                # Filter rows where is the required reportDate accessionNumber and primaryDocument
                df_r = df_r[df_r['reportDate'] == reporting_date][['accessionNumber', 'primaryDocument']]
                
                # Remove leading zeros the url needs to get rid of the leading zeros  
                cik_url = cik.lstrip('0')
                
                # The url works without hyphens in the accessionNumber
                accessionNumber = df_r['accessionNumber'].iloc[0].replace('-', '')
                primaryDocument = df_r['primaryDocument'].iloc[0]
                
                # Join the url string
                document_url = [cik, reporting_date,
                                f'https://www.sec.gov/Archives/edgar/data/{cik_url}/{accessionNumber}/{primaryDocument}'
                               ]
                
                return document_url
            
            # Return a list of Nones if the document is not found
            else:
                return [None, None, None]
            
        elif 'reportDate' in df_files.columns:
            
            # Select only the 10-K and the 10-Q case insensitive
            df_f = df_files[df_files['form'].str.lower().isin(['10-k', '10-q'])]
            
            if not df_f.empty and reporting_date in df_f['reportDate'].values:
                
                # Filter rows where is the required reportDate accessionNumber and primaryDocument
                df_f = df_f[df_f['reportDate'] == reporting_date][['accessionNumber', 'primaryDocument']]
                
                
                # Remove leading zeros the url needs to get rid of the leading zeros                
                cik_url = cik.lstrip('0')
                
                # The url works without hyphens in the accessionNumber
                accessionNumber = df_f['accessionNumber'].iloc[0].replace('-', '')
                primaryDocument = df_f['primaryDocument'].iloc[0]
                
                # Join the url string
                document_url = [cik, reporting_date,
                                f'https://www.sec.gov/Archives/edgar/data/{cik_url}/{accessionNumber}/{primaryDocument}'
                               ]
                
                return document_url
            
            # Return a list of Nones if the document is not found
            else:
                return [None, None, None]
        
        # Return a list of Nones if the document is not found
        else:
            return [None, None, None]

## Test the function

In [11]:
print(get_document_url('0000049196', '2016-12-31'))

[None, None, None]


In [12]:
# Create a list to loop and apply the function
companies_list = companies_df[['cik', 'reporting_date']].values.tolist()
print(companies_list)

[['0000789019', '2022-03-31'], ['0001304421', '2017-06-30'], ['0001411579', '2016-12-31'], ['0000001800', '2017-03-31'], ['0001287750', '2016-12-31'], ['0000827054', '2016-12-31'], ['0001004702', '2016-03-31'], ['0000097745', '2016-10-01'], ['0000707179', '2016-12-31'], ['0000049196', '2016-12-31'], ['0001115055', '2016-06-30'], ['0001262823', '2016-09-30'], ['0000715957', '2016-09-30'], ['0000001800', '2017-09-30'], ['0001039399', '2016-06-25']]


In [13]:
# Get the submission url of the list of ciks
documents_urls = []

# Use tqdm to have a progress bar for long jobs
for index, sublist in enumerate(tqdm(companies_list)):
    # Extract the cik
    cik = sublist[0]
    
    # Extract the effective date
    reporting_date = sublist[1]
    
    if cik:
        documents_urls.append(get_document_url(cik, reporting_date))
    else:
        documents_urls.append([None, None, None])
    
    # Check if the current iteration is a multiple of 9 less that 10
    if index % 9 == 0:
        # Delay for 1 seconds to avoid the downloading limits of the API
        time.sleep(1)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:09<00:00,  1.56it/s]


In [14]:
# Now there is a list of lists with the urls and other infomation
for doc in documents_urls:
    print(doc)

['0000789019', '2022-03-31', 'https://www.sec.gov/Archives/edgar/data/789019/000156459022015675/msft-10q_20220331.htm']
['0001304421', '2017-06-30', 'https://www.sec.gov/Archives/edgar/data/1304421/000155837017005955/cnsl-20170630x10q.htm']
['0001411579', '2016-12-31', 'https://www.sec.gov/Archives/edgar/data/1411579/000141157917000021/amc-20161231x10k.htm']
['0000001800', '2017-03-31', 'https://www.sec.gov/Archives/edgar/data/1800/000110465917029430/a17-8899_110q.htm']
['0001287750', '2016-12-31', 'https://www.sec.gov/Archives/edgar/data/1287750/000104746917000808/a2230898z10-k.htm']
['0000827054', '2016-12-31', 'https://www.sec.gov/Archives/edgar/data/827054/000082705417000018/a1231201610q.htm']
['0001004702', '2016-03-31', 'https://www.sec.gov/Archives/edgar/data/1004702/000119312516583947/d167739d10q.htm']
['0000097745', '2016-10-01', 'https://www.sec.gov/Archives/edgar/data/97745/000009774516000072/tmoq3201610q.htm']
[None, None, None]
[None, None, None]
['0001115055', '2016-06-30

## There was two documents that couldn't be found, the structure is complex, my advise is check all the json files that are downloaded to understad better the way the information is stored.

In [15]:
# Create a new column 'url' in companies_df
companies_df['url'] = None  # Initialize the column

# Iterate over each row in companies_df
for index, row in companies_df.iterrows():
    cik = row['cik']
    reporting_date = row['reporting_date']
    
    # Iterate over the list store the url where cik and reporting_date match
    for sublist in documents_urls:
        if cik == sublist[0] and reporting_date == sublist[1]:
            companies_df['url'].iloc[index] = sublist[2] 

companies_df.head(15)

,form_type,filing_date,reporting_date,ticker,cik,url
0,10-Q,2022-04-26,2022-03-31,MSFT,0000789019,https://www.sec.gov/Archives/edgar/data/789019...
1,10-Q,2017-08-04,2017-06-30,CNSL,0001304421,https://www.sec.gov/Archives/edgar/data/130442...
2,10-K,2017-03-10,2016-12-31,AMC,0001411579,https://www.sec.gov/Archives/edgar/data/141157...
3,10-Q,2017-05-03,2017-03-31,ABT,0000001800,https://www.sec.gov/Archives/edgar/data/1800/0...
4,10-K,2017-02-22,2016-12-31,ARCC,0001287750,https://www.sec.gov/Archives/edgar/data/128775...
5,10-Q,2017-02-08,2016-12-31,MCHP,0000827054,https://www.sec.gov/Archives/edgar/data/827054...
6,10-Q,2016-05-09,2016-03-31,OCFC,0001004702,https://www.sec.gov/Archives/edgar/data/100470...
7,10-Q,2016-11-04,2016-10-01,TMO,0000097745,https://www.sec.gov/Archives/edgar/data/97745/...
8,10-k,2017-02-16,2016-12-31,ONB,0000707179,None
9,10-K,2018-07-30,2016-12-31,HBAN,0000049196,None


In [16]:
# Save the DataFrame to a CSV file and access the documents form the url link
companies_df.to_csv('companies_urls.csv', index=False)